In [1]:
import pandas as pd
import os
import numpy as np

In [13]:
path = "C:/Users/doosti/Dropbox (Chapman)/Research/Research Projects/Fitness/Data/"
file = "labeled_comments.csv"

In [42]:
comments_labeled = pd.read_csv(os.path.join(path, file))
comments = pd.read_csv(os.path.join(path, "merged_comments.csv"))
videos = pd.read_csv(os.path.join(path, "videos_101223.csv"))
# matching with the creator id
comments['creator_id'] = comments.merge(videos,how='left',on='video_id').creator_id.values
comments_labeled['creator_id'] = comments_labeled.merge(videos,how='left',on='video_id').creator_id.values

C:\Users\doosti\AppData\Local\Temp\ipykernel_188996\3935688471.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  comments_labeled = pd.read_csv(os.path.join(path, file))


809335 827580


In [43]:
print((comments.creator_id==comments_labeled.creator_id).sum(),comments.shape[0])
print(comments.isnull().sum())
print(comments_labeled.isnull().sum())

809335 827580
video_id             0
user_id              0
comment_text       100
no_likes            14
no_replies          14
comment_year        14
comment_month       28
comment_day         28
creator_id       18245
dtype: int64
video_id              0
user_id               0
comment_text        100
no_likes             14
no_replies           14
comment_year         14
comment_month        28
comment_day          28
label            764525
label_score      764525
creator_id        18245
dtype: int64


In [18]:
print(comments_labeled.columns)
print(comments.columns)

Index(['video_id', 'user_id', 'comment_text', 'no_likes', 'no_replies',
       'comment_year', 'comment_month', 'comment_day', 'label', 'label_score'],
      dtype='object')
Index(['video_id', 'user_id', 'comment_text', 'no_likes', 'no_replies',
       'comment_year', 'comment_month', 'comment_day', 'creator_id'],
      dtype='object')


In [44]:
# threshold = 11
comments_labeled = comments_labeled[comments_labeled.video_id.str.len() == 11].copy()
print(comments_labeled.shape)
# drop rows with missing year, month, or day
comments_labeled = comments_labeled.dropna(subset=['comment_year', 'comment_month', 'comment_day'])
print(comments_labeled.shape)
print(comments_labeled.isnull().sum())

(827566, 11)
(827552, 11)
video_id              0
user_id               0
comment_text        100
no_likes              0
no_replies            0
comment_year          0
comment_month         0
comment_day           0
label            764497
label_score      764497
creator_id        18231
dtype: int64


In [48]:
# Create a function to format and convert the date
def to_datetime_with_padding(year, month, day):
    # Pad month and day with zeros if necessary
    month = f"{month:02d}"
    day = f"{day:02d}"
    # Combine and convert to datetime
    date_str = f"{year}-{month}-{day}"
    return pd.to_datetime(date_str)

comments_labeled['comment_year'] = comments_labeled.comment_year.astype(int)
comments_labeled['comment_month'] = comments_labeled.comment_month.astype(int)
comments_labeled['comment_day'] = comments_labeled.comment_day.astype(int)

# Apply the function to each row
comments_labeled['date'] = comments_labeled.apply(lambda row: to_datetime_with_padding(row["comment_year"], row["comment_month"], row["comment_day"]), axis=1)


In [52]:

channel_day = comments_labeled.groupby(['creator_id','date']).label.value_counts().unstack().reset_index()
channel_day = channel_day.fillna(0)
channel_day.head()

label,creator_id,date,appearance,competence,fitness,interest,social
0,262h9yTG1f03Nobo501,2020-11-11,0.0,3.0,0.0,0.0,0.0
1,262h9yTG1f03Nobo501,2020-11-12,0.0,9.0,0.0,0.0,0.0
2,262h9yTG1f03Nobo501,2020-11-13,0.0,6.0,1.0,1.0,0.0
3,262h9yTG1f03Nobo501,2020-11-14,0.0,11.0,0.0,1.0,0.0
4,262h9yTG1f03Nobo501,2020-11-15,0.0,11.0,2.0,0.0,0.0


In [53]:
channel_day.to_csv(os.path.join(path, "channel_day.csv"), index=False)
